In [ ]:
%matplotlib widget
import nest
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from nest import raster_plot

In [ ]:
data = np.load('/home/justidy1/LabCode/IctalPlaceCell/data/line_maze_data.npz', allow_pickle=True)

def stim_cue_net(conn_CA3_to_CA3,conn_CA3_to_CA1,
                 sim_time,cue_location,cue_time,cue_duration=20,cue_rate=30,cue_weight=50,
                 seed=1234):
    # making a new newtork
    nest.ResetKernel()
    nest.local_num_threads = 16
    nest.rng_seed=seed

    # Create CA3 and CA1 populations of adaptive exponential integrate-and-fire neurons
    N_pyr = 1250
    tau_decay_pyr_e = 9.0
    tau_rise_pyr_e = 1.0
    tau_decay_pyr_i = 3.0
    tau_rise_pyr_i = 0.3
    tau_decay_gc = 5.4
    tau_rise_gc = 0.65

    pyr_spec = {'C_m':180,'g_L':4.3,'E_L':-75,'Delta_T':4.23,'V_th':-24,'V_peak':-3.25,'V_reset':-29.7,'t_ref':5.9,
                'tau_w':84.93,'a':-0.27,'b':206.84,'E_rev':[0,-90,0],
                'tau_rise':[tau_rise_pyr_e,tau_rise_pyr_i,tau_rise_gc],'tau_decay':[tau_decay_pyr_e,tau_decay_pyr_i,tau_decay_gc]}
    CA3_pyr_aeif = nest.Create("aeif_cond_beta_multisynapse", N_pyr, pyr_spec)
    CA1_pyr_aeif = nest.Create("aeif_cond_beta_multisynapse", N_pyr, pyr_spec)
    # make interneurons in CA3 and CA1
    N_int = 250
    int_spec = {'C_m':118,'g_L':7.5,'E_L':-74,'Delta_T':4.6,'V_th':-57.7,'V_peak':-34.78,'V_reset':-65,'t_ref':1,
                'tau_w':178.58,'a':3.05,'b':0.91,'E_rev':[0,-90],
                'tau_rise':[tau_rise_pyr_e,tau_rise_pyr_i],'tau_decay':[tau_decay_pyr_e,tau_decay_pyr_i]}
    CA3_int_aeif = nest.Create("aeif_cond_beta_multisynapse", N_int, int_spec)
    CA1_int_aeif = nest.Create("aeif_cond_beta_multisynapse", N_int, int_spec)

    # connect the CA3 and CA1 populations with static synapses with the weights learned from the previous simulation
    # the order of nodes is presevered since the neurons are created in the same order
    nest.Connect(conn_CA3_to_CA3['source'], conn_CA3_to_CA3['target'], 
                'one_to_one', syn_spec={'synapse_model':'static_synapse','weight': np.array(conn_CA3_to_CA3['weight']), 'receptor_type':1})
    nest.Connect(conn_CA3_to_CA1['source'], conn_CA3_to_CA1['target'], 
                'one_to_one', syn_spec={'synapse_model':'static_synapse','weight': np.array(conn_CA3_to_CA1['weight']), 'receptor_type':1})

    # now we wire pyr to int with a probability of 0.1
    nest.Connect(CA3_pyr_aeif, CA3_int_aeif, syn_spec={'weight': 0.85, 'receptor_type':1}, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.1,'allow_autapses': False})
    nest.Connect(CA1_pyr_aeif, CA1_int_aeif, syn_spec={'weight': 0.85, 'receptor_type':1}, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.1,'allow_autapses': False})

    # connect the interneurons with each other with a probability of 0.25
    nest.Connect(CA3_int_aeif, CA3_int_aeif, syn_spec={'weight': 5, 'receptor_type':2}, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.25,'allow_autapses': False})
    nest.Connect(CA1_int_aeif, CA1_int_aeif, syn_spec={'weight': 5, 'receptor_type':2}, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.25,'allow_autapses': False})

    # and the interneurons to the pyramidal cells with a probability of 0.25
    nest.Connect(CA3_int_aeif, CA3_pyr_aeif, syn_spec={'weight': 0.6, 'receptor_type':2}, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.25,'allow_autapses': False})
    nest.Connect(CA1_int_aeif, CA1_pyr_aeif, syn_spec={'weight': 0.6, 'receptor_type':2}, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.25,'allow_autapses': False})

    # create a stim to drive all the CA3 pyramidal cells
    stim = nest.Create("poisson_generator",1,{'rate': 12.0})
    nest.Connect(stim, CA3_pyr_aeif, syn_spec={'weight': 20.0, 'receptor_type':3}, conn_spec={'rule': 'all_to_all'})

    # create a cue to drive a subset of the CA3 pyramidal cells
    cue = nest.Create("inhomogeneous_poisson_generator",1,{'rate_times': [1,cue_time,cue_time+cue_duration],'rate_values': [0,cue_rate,0]})
    nest.Connect(cue, CA3_pyr_aeif[cue_location], syn_spec={'weight': cue_weight, 'receptor_type':3}, conn_spec={'rule': 'all_to_all'})

    # monitor the spiking activity of the CA3 and CA1 populations
    spike_detector = nest.Create("spike_recorder")
    nest.Connect(CA3_pyr_aeif, spike_detector)
    nest.Connect(CA1_pyr_aeif, spike_detector)
    nest.Connect(CA3_int_aeif, spike_detector)
    nest.Connect(CA1_int_aeif, spike_detector)

    # compute the CA3 and CA1 LFP
    CA3_multimeter = nest.Create("multimeter",1,{'record_from':['V_m','g_1','g_2','g_3'],'interval':1})
    CA1_multimeter = nest.Create("multimeter",1,{'record_from':['V_m','g_1','g_2'],'interval':1})
    nest.Connect(CA3_multimeter, CA3_pyr_aeif, conn_spec={'rule': 'fixed_outdegree', 'outdegree': 200,'allow_multapses': False})
    nest.Connect(CA1_multimeter, CA1_pyr_aeif, conn_spec={'rule': 'fixed_outdegree', 'outdegree': 200,'allow_multapses': False})

    # simulate the network
    nest.Simulate(sim_time)

    # get the spikes
    spikes = nest.GetStatus(spike_detector,keys='events')[0]

    # get the LFP
    CA3_recordings = nest.GetStatus(CA3_multimeter,keys='events')[0]
    CA1_recordings = nest.GetStatus(CA1_multimeter,keys='events')[0]
                                                                          
    CA3_lfp = CA3_recordings['g_1']*(CA3_recordings['V_m']-0) + CA3_recordings['g_2']*(CA3_recordings['V_m']--90) + CA3_recordings['g_3']*(CA3_recordings['V_m']-0) #pA
    CA1_lfp = CA1_recordings['g_1']*(CA1_recordings['V_m']-0) + CA1_recordings['g_2']*(CA1_recordings['V_m']--90) # pA
    CA3_lfp *= (1/(4*np.pi*0.3*5e-6))*1e-9 #mV
    CA1_lfp *= (1/(4*np.pi*0.3*5e-6))*1e-9

    return spikes,CA3_lfp,CA1_lfp,CA3_recordings['times'],CA1_recordings['times'],CA3_recordings['senders'],CA1_recordings['senders']

def placerange_to_indexes(xlow,xhigh,pf):
    return np.argwhere((pf>xlow) & (pf<xhigh)).flatten()

def sim_wrapper(data,wh,ictal,sim_time,cue_locations,cue_time,cue_duration=20,cue_rate=30,cue_weight=50,seed=1234):
    if ictal:
        conn_CA3_to_CA3=data['conn_CA3_to_CA3_ictal'][wh]
        conn_CA3_to_CA1=data['conn_CA3_to_CA1_ictal'][wh]
        pf_CA3 = data['all_pf_CA3_ictal'][wh]
        pf_CA1 = data['all_pf_CA1_ictal'][wh]
    else:
        conn_CA3_to_CA3=data['conn_CA3_to_CA3_controls'][wh]
        conn_CA3_to_CA1=data['conn_CA3_to_CA1_controls'][wh]
        pf_CA3 = data['all_pf_CA3_controls'][wh]
        pf_CA1 = data['all_pf_CA1_controls'][wh]
    
    spikes,CA3_lfp,CA1_lfp,CA3_lfp_t,CA1_lfp_t,CA3_lfp_s,CA1_lfp_s = stim_cue_net(conn_CA3_to_CA3,conn_CA3_to_CA1,
                                                                                         sim_time,placerange_to_indexes(cue_locations[0],cue_locations[1],pf_CA3),
                                                                                         cue_time,cue_duration,cue_rate,cue_weight,seed)
    lfp_sig_ts = np.arange(0,np.max(CA3_lfp_t),1)+1
    CA3_lfp_sig = np.zeros(len(lfp_sig_ts))
    CA1_lfp_sig = np.zeros(len(lfp_sig_ts))
    CA3_senders = np.unique(CA3_lfp_s)
    CA1_senders = np.unique(CA1_lfp_s)

    for send in CA3_senders:
        # interpolate the LFP signal
        f = interpolate.interp1d(CA3_lfp_t[CA3_lfp_s==send],CA3_lfp[CA3_lfp_s==send],fill_value='extrapolate')
        CA3_lfp_sig += f(lfp_sig_ts)

    for send in CA1_senders:
        # interpolate the LFP signal
        f = interpolate.interp1d(CA1_lfp_t[CA1_lfp_s==send],CA1_lfp[CA1_lfp_s==send],fill_value='extrapolate')
        CA1_lfp_sig += f(lfp_sig_ts)

    s_times = spikes['times']
    s_senders = spikes['senders']
    s_place_CA1 = np.zeros(len(s_times))
    s_place_CA3 = np.zeros(len(s_times))

    for i in range(len(s_times)):
        if s_senders[i] < len(pf_CA1)+1250 and s_senders[i] >= 1250:
            s_place_CA1[i] = pf_CA1[s_senders[i]-1250]
            s_place_CA3[i] = np.nan
        elif s_senders[i] < len(pf_CA3):
            s_place_CA3[i] = pf_CA3[s_senders[i]]
            s_place_CA1[i] = np.nan
        else:
            s_place_CA1[i] = np.nan
            s_place_CA3[i] = np.nan

    return s_times,s_senders,s_place_CA1,s_place_CA3,CA3_lfp_sig,CA1_lfp_sig,lfp_sig_ts

In [ ]:
ictal=False
wh=0
s_times,s_senders,s_place_CA1,s_place_CA3,CA3_lfp_sig,CA1_lfp_sig,lfp_sig_ts = sim_wrapper(data,wh,ictal,5000,[-60,-40],2000,cue_weight=50,seed=18)

plt.close('all')
plt.figure()
plt.plot(s_times,s_place_CA3,'.')
plt.plot(s_times,s_place_CA1,'.')
plt.show()

In [ ]:
plt.close('all')
plt.figure()
plt.scatter(s_times,s_senders,marker='|')
plt.show()

In [ ]:
plt.close('all')
plt.figure()
plt.plot(lfp_sig_ts,CA1_lfp_sig)
plt.show()

In [ ]:
from scipy import signal
import pywt

fs = 1000
freqs = np.logspace(1.2,2.4,200)/fs
scale = pywt.frequency2scale('cmor1.5-1.0', freqs)

coef, freqs=pywt.cwt(CA1_lfp_sig,scale,'cmor1.5-1.0',1/fs)

plt.close('all')
plt.figure()
plt.pcolormesh(lfp_sig_ts,freqs,np.abs(coef))
#plt.yscale('log')
plt.show()

In [ ]:
butt_lp = signal.butter(7, 80, 'low', fs=fs, output='sos')
CA1_lfp_sig_filt = signal.sosfilt(butt_lp,CA1_lfp_sig)
peaks,_=signal.find_peaks(-CA1_lfp_sig_filt,prominence=1,distance=50)
# for each peak, grab the lfp between -wid and wid ms
wid = 150
each_osc = []
t_osc = []
Pxx_osc = []
f_osc = []
for i in range(len(peaks)):
    left_idx = np.argmin(np.abs(lfp_sig_ts-(lfp_sig_ts[peaks[i]]-wid)))
    right_idx = np.argmin(np.abs(lfp_sig_ts-(lfp_sig_ts[peaks[i]]+wid)))
    each_osc.append(CA1_lfp_sig[left_idx:right_idx])

    # the time, centered around the peak
    t_osc.append(lfp_sig_ts[left_idx:right_idx]-lfp_sig_ts[peaks[i]])

    # welch spectrum
    f, Pxx = signal.welch(CA1_lfp_sig[left_idx:right_idx],fs=fs,nperseg=256,noverlap=32,nfft=1024)
    Pxx_osc.append(Pxx)
    f_osc.append(f)


plt.close('all')
plt.figure()
plt.plot(lfp_sig_ts,CA1_lfp_sig)
plt.plot(lfp_sig_ts[peaks],CA1_lfp_sig_filt[peaks],'x')
plt.show()

In [ ]:
plt.close('all')
plt.figure()
for i in range(len(each_osc)):
    plt.plot(t_osc[i],each_osc[i])
plt.show()

In [ ]:
plt.close('all')
plt.figure()
mean_Pxx = np.mean(Pxx_osc,axis=0)
std_Pxx = np.std(Pxx_osc,axis=0)
plt.plot(f_osc[0],mean_Pxx,'.')
plt.fill_between(f_osc[0],mean_Pxx-std_Pxx,mean_Pxx+std_Pxx,alpha=0.5)
plt.yscale('log')
#plt.xlim([25,160])
plt.show()